In [2]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

# **Objetivo**
En este programa se hará una red neuronal que reciba como input el coeficiente de absorción y como output tengamos radio, fracción volumen y espesor de la película

### Primero juntaremos todos los archivos en uno solo

In [3]:
files = [file for file in os.listdir('./parametros')]

param = pd.DataFrame()
for file in files:
  df = pd.read_csv('./parametros/'+file)
  param = pd.concat([param, df], axis= 0)

Imprimimos la base de datos creada anteriormente para corroborar que se haya ejecutado como deseábamos la línea anterior.

In [4]:
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,301,0.016685,1,10,100
1,302,0.016167,1,10,100
2,303,0.015703,1,10,100
3,304,0.015290,1,10,100
4,305,0.014916,1,10,100
...,...,...,...,...,...
19195,896,0.002255,1,9,120
19196,897,0.002257,1,9,120
19197,898,0.002258,1,9,120
19198,899,0.002260,1,9,120


### Revolvemos los datos
Esto es para que el entrenamiento de la red sea óptimo y aís poderle dar una mayor gama de datos.

In [5]:
from sklearn.model_selection import train_test_split
param = param.sample(frac=1, random_state=42).reset_index(drop=True)
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,753,0.001533,1,23,120
1,310,0.014692,3,10,120
2,625,0.002815,1,10,85
3,604,0.000572,1,9,120
4,373,0.003104,1,21,120
...,...,...,...,...,...
38395,566,0.000711,1,10,70
38396,785,0.000418,12,10,120
38397,659,0.000772,1,9,120
38398,561,0.005686,1,10,105


## Separamos nuestra base de datos revuelta en dos
Una que servirá para entrenar el modelo y otra para comprobar que el modelo tenga un funcionamiento optimo. Esta división será de 80-20 respectivamente.

In [6]:
train_df, test_df = train_test_split(param, test_size=0.2, random_state=42)
#test será el 20% de los datos mientras train el 80%
test_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
27452,492,0.006226,5,10,120
15009,831,0.001993,1,26,120
8124,700,0.000512,1,26,120
38103,748,0.000418,11,10,120
25872,432,0.002877,1,10,65
...,...,...,...,...,...
3746,820,0.002003,7,10,120
18195,422,0.000481,1,7,120
37286,551,0.033743,4,10,120
15361,703,0.000239,1,29,120


In [7]:
train_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
36772,825,0.001564,11,10,120
22714,538,0.004552,1,10,65
3031,442,0.000189,1,10,120
9632,391,0.000021,1,10,115
29786,870,0.002197,1,11,120
...,...,...,...,...,...
6265,760,0.002772,1,10,100
11284,623,0.001487,1,32,120
38158,836,0.001522,1,10,135
860,324,0.008735,1,6,120


# Comenzamos a hacer la red neuronal


In [77]:
# Cargamos el DataFrame de entrenamiento
df = train_df
# Separarmos los datos de entrada y salida
X = df['Índice de absorción']
Y = df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [78]:
# Convertimos los dataframes en numpy arrays, esto para que su manejo sea más sencillo
X = np.array(X).reshape(-1, 1)
y = np.array(Y)

In [79]:
# Creamos el modelo de la red neuronal
model = Sequential()
model.add(Dense(64, input_dim=1, activation='relu'))
model.add(Dense(3, activation='linear'))

In [80]:
# Asignamos el tipo de error y el optimizador que vamos a utilizar
model.compile(loss='mean_absolute_error', optimizer='adam')

In [83]:
# Entrenamos a la red neuronal, en este caso lo hicimos en 100 épocas
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
768/768 [==============================] - 2s 1ms/step - loss: 38.6021 - val_loss: 30.8146
Epoch 2/10
768/768 [==============================] - 1s 1ms/step - loss: 17.4660 - val_loss: 4.9069
Epoch 3/10
768/768 [==============================] - 1s 963us/step - loss: 4.8167 - val_loss: 4.8251
Epoch 4/10
768/768 [==============================] - 1s 1ms/step - loss: 4.7701 - val_loss: 4.7816
Epoch 5/10
768/768 [==============================] - 1s 926us/step - loss: 4.7341 - val_loss: 4.7576
Epoch 6/10
768/768 [==============================] - 1s 954us/step - loss: 4.7286 - val_loss: 4.7606
Epoch 7/10
768/768 [==============================] - 1s 928us/step - loss: 4.7282 - val_loss: 4.7707
Epoch 8/10
768/768 [==============================] - 1s 1ms/step - loss: 4.7275 - val_loss: 4.7584
Epoch 9/10
768/768 [==============================] - 1s 988us/step - loss: 4.7274 - val_loss: 4.7534
Epoch 10/10
768/768 [==============================] - 1s 1ms/step - loss: 4.7271 - val

In [84]:
# Hacemos las predicciones y las imprimimos
arr = test_df['Índice de absorción'].to_numpy()
y_pred = model.predict(arr)
print(y_pred)

240/240 [==============================] - 0s 882us/step
[[  1.010767   10.024649  120.0034   ]
 [  1.0095743  10.026845  120.00331  ]
 [  1.0091568  10.027614  120.00329  ]
 ...
 [  1.0185198  10.010371  120.004    ]
 [  1.0090802  10.027755  120.003265 ]
 [  1.010771   10.02464   120.0034   ]]


In [85]:
# Almacenamos estas predicciones en un nuevo DataFrame
predicciones = pd.DataFrame(y_pred)
predicciones

,0,1,2
0,1.010767,10.024649,120.003403
1,1.009574,10.026845,120.003311
2,1.009157,10.027614,120.003288
3,1.009131,10.027661,120.003288
4,1.009823,10.026387,120.003342
...,...,...,...
7675,1.009577,10.026840,120.003319
7676,1.009148,10.027629,120.003288
7677,1.018520,10.010371,120.003998
7678,1.009080,10.027755,120.003265


In [86]:
# Renombramos las columnas para que equipare a la sintaxis de las bases de datos anteriores
predicciones.columns = ['Fracción Volumen', 'Radio (nm)', 'Espesor de la película (nm)']
predicciones

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.010767,10.024649,120.003403
1,1.009574,10.026845,120.003311
2,1.009157,10.027614,120.003288
3,1.009131,10.027661,120.003288
4,1.009823,10.026387,120.003342
...,...,...,...
7675,1.009577,10.026840,120.003319
7676,1.009148,10.027629,120.003288
7677,1.018520,10.010371,120.003998
7678,1.009080,10.027755,120.003265


### Redondeando los valores

In [87]:
redondeo_arriba = predicciones.round()
redondeo_arriba

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.0,10.0,120.0
1,1.0,10.0,120.0
2,1.0,10.0,120.0
3,1.0,10.0,120.0
4,1.0,10.0,120.0
...,...,...,...
7675,1.0,10.0,120.0
7676,1.0,10.0,120.0
7677,1.0,10.0,120.0
7678,1.0,10.0,120.0


In [88]:
redondeo_ceil =  predicciones.applymap(lambda x: math.ceil(x))
redondeo_ceil = redondeo_ceil.astype(float)
redondeo_ceil

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,2.0,11.0,121.0
1,2.0,11.0,121.0
2,2.0,11.0,121.0
3,2.0,11.0,121.0
4,2.0,11.0,121.0
...,...,...,...
7675,2.0,11.0,121.0
7676,2.0,11.0,121.0
7677,2.0,11.0,121.0
7678,2.0,11.0,121.0


In [89]:
redondeo_abajo = predicciones.apply(np.floor)
redondeo_abajo

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.0,10.0,120.0
1,1.0,10.0,120.0
2,1.0,10.0,120.0
3,1.0,10.0,120.0
4,1.0,10.0,120.0
...,...,...,...
7675,1.0,10.0,120.0
7676,1.0,10.0,120.0
7677,1.0,10.0,120.0
7678,1.0,10.0,120.0


# Probando la precisión del modelo
Primero lo haremos por cada parámetro. Se le restará los valores predichos por la red a los valores originales del dataframe test. Se contarán el número de ceros y ese será lo acertada que es nuestra red. De tener valores inferiores a 80% en número de ceros este rango se incrementará hasta llegar a un valor que el 80% de los valores sean menores a este.

## Fracción de Volumen

Calculando con el redondeo hacia abajo

In [90]:
redondeo_abajo_fracvol = redondeo_abajo['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_abajo_fracvol - test_df_fracvol
# count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en fracción volumen")

presición del  76.30208333333333 %. Con el redondeo para abajo en fracción volumen


In [20]:
num_zeros # Checamos el número de ceros que tenemos

3394

Umbral de 1

In [91]:
# Quiere decir que el 79.60% de los datos son (valor absoluto) 1 o 0 después de restar el test con el predicciones
umbral = 1
num = np.count_nonzero(np.abs(dif) <= umbral)
porcentaje = num*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en fracción volumen y un rango de 1")

presición del  77.95572916666667 %. Con el redondeo para abajo en fracción volumen y un rango de 1


In [92]:
nums = []
for num in dif:
    if np.abs(num) <= 1:
        nums.append(num)
len(nums)

5987

Calculando con el redondeo hacia arriba

In [93]:
redondeo_arriba_fracvol = redondeo_arriba['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_arriba_fracvol - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para arriba en fracción volumen")

presición del  76.30208333333333 %. Con el redondeo para arriba en fracción volumen


In [94]:
num_zeros # Checamos el número de ceros que tenemos

5860

In [95]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en fracción volumen y un rango de", umbral)
    i = i+1

presición del  77.95572916666667 %. Con el redondeo para arriba en fracción volumen y un rango de 1
presición del  79.55729166666667 %. Con el redondeo para arriba en fracción volumen y un rango de 2
presición del  81.22395833333333 %. Con el redondeo para arriba en fracción volumen y un rango de 3


Calculando con celi

In [96]:
redondeo_ceil_fracvol = redondeo_ceil['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_ceil_fracvol - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo con ceil en fracción volumen")

presición del  1.6536458333333333 %. Con el redondeo con ceil en fracción volumen


In [97]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo ceil en fracción volumen y un rango de", umbral)
    i = i+1

presición del  79.55729166666667 %. Con el redondeo ceil en fracción volumen y un rango de 1
presición del  81.22395833333333 %. Con el redondeo ceil en fracción volumen y un rango de 2


# Radio

Calculando con el redondeo hacia abajo

In [98]:
redondeo_abajo_radio = redondeo_abajo['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_abajo_radio - test_df_radio
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en radio")

presición del  51.419270833333336 %. Con el redondeo para abajo en radio


In [99]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para abajo en radio y un rango de", umbral)
    i = i+1

presición del  55.234375 %. Con el redondeo para abajo en radio y un rango de 1
presición del  58.138020833333336 %. Con el redondeo para abajo en radio y un rango de 2
presición del  61.393229166666664 %. Con el redondeo para abajo en radio y un rango de 3
presición del  64.62239583333333 %. Con el redondeo para abajo en radio y un rango de 4
presición del  67.8515625 %. Con el redondeo para abajo en radio y un rango de 5
presición del  69.58333333333333 %. Con el redondeo para abajo en radio y un rango de 6
presición del  71.2109375 %. Con el redondeo para abajo en radio y un rango de 7
presición del  72.87760416666667 %. Con el redondeo para abajo en radio y un rango de 8
presición del  74.27083333333333 %. Con el redondeo para abajo en radio y un rango de 9
presición del  75.6640625 %. Con el redondeo para abajo en radio y un rango de 10
presición del  77.27864583333333 %. Con el redondeo para abajo en radio y un rango de 11
presición del  78.90625 %. Con el redondeo para abajo en 

Calculando con el redondeo hacia arriba

In [100]:
redondeo_arriba_radio = redondeo_arriba['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_arriba_radio - test_df_radio
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio")

presición del  52.265625 %. Con el redondeo para arriba en radio


In [101]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio y un rango de", umbral)
    i = i+1

presición del  55.234375 %. Con el redondeo para arriba en radio y un rango de 1
presición del  58.138020833333336 %. Con el redondeo para arriba en radio y un rango de 2
presición del  61.393229166666664 %. Con el redondeo para arriba en radio y un rango de 3
presición del  64.62239583333333 %. Con el redondeo para arriba en radio y un rango de 4
presición del  67.8515625 %. Con el redondeo para arriba en radio y un rango de 5
presición del  69.58333333333333 %. Con el redondeo para arriba en radio y un rango de 6
presición del  71.2109375 %. Con el redondeo para arriba en radio y un rango de 7
presición del  72.87760416666667 %. Con el redondeo para arriba en radio y un rango de 8
presición del  74.27083333333333 %. Con el redondeo para arriba en radio y un rango de 9
presición del  75.6640625 %. Con el redondeo para arriba en radio y un rango de 10
presición del  77.27864583333333 %. Con el redondeo para arriba en radio y un rango de 11
presición del  78.90625 %. Con el redondeo par

Calculando con celi

In [102]:
redondeo_ceil_radio = redondeo_ceil['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_ceil_radio - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo el ceil en radio")

presición del  1.7708333333333333 %. Con el redondeo el ceil en radio


In [103]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio y un rango de", umbral)
    i = i+1

presición del  4.791666666666667 %. Con el redondeo para arriba en radio y un rango de 1
presición del  8.020833333333334 %. Con el redondeo para arriba en radio y un rango de 2
presición del  11.09375 %. Con el redondeo para arriba en radio y un rango de 3
presición del  14.127604166666666 %. Con el redondeo para arriba en radio y un rango de 4
presición del  17.135416666666668 %. Con el redondeo para arriba en radio y un rango de 5
presición del  18.776041666666668 %. Con el redondeo para arriba en radio y un rango de 6
presición del  20.442708333333332 %. Con el redondeo para arriba en radio y un rango de 7
presición del  22.044270833333332 %. Con el redondeo para arriba en radio y un rango de 8
presición del  23.697916666666668 %. Con el redondeo para arriba en radio y un rango de 9
presición del  100.0 %. Con el redondeo para arriba en radio y un rango de 10


# Espesor

Calculando con el redondeo hacia abajo

In [104]:
redondeo_abajo_espesor = redondeo_abajo['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_abajo_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor")

presición del  76.10677083333333 %. Con el redondeo para abajo en espesor


In [105]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 3
presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 4
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 8
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 9
presición del  82.60416666666667 %. Con el redondeo para abajo en espesor y un rango de 10


Calculando con el redondeo hacia arriba

In [106]:
redondeo_arriba_espesor = redondeo_arriba['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_arriba_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor")

presición del  76.10677083333333 %. Con el redondeo para abajo en espesor


In [107]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 3
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 4
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 8
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 9
presición del  82.60416666666667 %. Con el redondeo para arriba en espesor y un rango de 10


Calculando con celi

In [108]:
redondeo_ceil_espesor = redondeo_ceil['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_ceil_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo ceil abajo en espesor")

presición del  0.0 %. Con el redondeo ceil abajo en espesor


In [39]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 3
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 4
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 8
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 9
presición del  82.60416666666667 %. Con el redondeo para arriba en espesor y un rango de 10


# Colectivo

Ajustando la columna test para poder tenerla igual que las de redondeo

In [40]:
test2 = test_df.drop(['Longitud de onda', 'Índice de absorción'], axis=1)
test2 = test2.astype(float)
nuevo = {'Fracvol': 'Fracción Volumen'}
test2 = test2.rename(columns = nuevo)
test2 = test2.reset_index()
test2 = test2.drop(['index'], axis = 1)
test2

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,5.0,10.0,120.0
1,1.0,26.0,120.0
2,1.0,26.0,120.0
3,11.0,10.0,120.0
4,1.0,10.0,65.0
...,...,...,...
7675,7.0,10.0,120.0
7676,1.0,7.0,120.0
7677,4.0,10.0,120.0
7678,1.0,29.0,120.0


In [41]:
dif_abajo = test2 - redondeo_abajo
dif_abajo

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
1,0.0,17.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
4,0.0,1.0,-54.0
...,...,...,...
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0
7677,0.0,1.0,1.0
7678,1.0,20.0,1.0


In [42]:
nans3 = ~(dif_abajo == 0).all(axis=1)
df_sin_ceros = dif_abajo[nans3]
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
1,0.0,17.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
4,0.0,1.0,-54.0
...,...,...,...
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0
7677,0.0,1.0,1.0
7678,1.0,20.0,1.0


In [43]:
df_sin_ceros.equals(dif_abajo)

True

Queremos que elimine los que tengan 2 0's

In [44]:
nans2 = dif_abajo
# Definimos una tolerancia
tolerancia = 1e-9
# Creamos una nueva columna que contenga el número de ceros por fila
nans2['zeros'] = (np.isclose(nans2, 0.0, atol=tolerancia)).sum(axis=1)
df_sin_ceros = nans2[nans2['zeros'] < 2]
df_sin_ceros = df_sin_ceros.drop(columns=['zeros'])
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
1,0.0,17.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
4,0.0,1.0,-54.0
...,...,...,...
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0
7677,0.0,1.0,1.0
7678,1.0,20.0,1.0


In [45]:
nans2.equals(df_sin_ceros)

False

Queremos que elimine los que tenga un 0

In [46]:
dif_abajo = test2 - redondeo_abajo

In [47]:
nan1 = dif_abajo[(dif_abajo != 0).all(1)]
nan1

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
5,1.0,1.0,1.0
6,1.0,1.0,11.0
...,...,...,...
7671,14.0,1.0,1.0
7674,1.0,16.0,1.0
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0


In [48]:
df_sin_ceros.equals(dif_abajo)

False

# Redondeo hacia arriba

In [49]:
dif_arriba = redondeo_arriba - test2
dif_arriba

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,-4.0,0.0,0.0
1,0.0,-16.0,0.0
2,0.0,-16.0,0.0
3,-10.0,0.0,0.0
4,0.0,0.0,55.0
...,...,...,...
7675,-6.0,0.0,0.0
7676,0.0,3.0,0.0
7677,1.0,0.0,0.0
7678,0.0,-19.0,0.0


Queremos que elimine los renglones que tengan 3 0's

In [50]:
nans3 = ~(dif_arriba == 0).all(axis=1)
df_sin_ceros = dif_arriba[nans3]
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,-4.0,0.0,0.0
1,0.0,-16.0,0.0
2,0.0,-16.0,0.0
3,-10.0,0.0,0.0
4,0.0,0.0,55.0
...,...,...,...
7675,-6.0,0.0,0.0
7676,0.0,3.0,0.0
7677,1.0,0.0,0.0
7678,0.0,-19.0,0.0


In [51]:
df_sin_ceros.equals(dif_arriba)

False

Queremos que ensenie los que tengan 2 0's

In [52]:
#Se imprimen las columnas que tengan solo 2 ceros
nans2 = dif_arriba
# Definimos una tolerancia
tolerancia = 1e-9
# Creamos una nueva columna que contenga el número de ceros por fila
nans2['zeros'] = (np.isclose(nans2, 0.0, atol=tolerancia)).sum(axis=1)
df_sin_ceros = nans2[nans2['zeros'] == 2]
df_sin_ceros = df_sin_ceros.drop(columns=['zeros'])
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,-4.0,0.0,0.0
1,0.0,-16.0,0.0
2,0.0,-16.0,0.0
3,-10.0,0.0,0.0
4,0.0,0.0,55.0
...,...,...,...
7675,-6.0,0.0,0.0
7676,0.0,3.0,0.0
7677,1.0,0.0,0.0
7678,0.0,-19.0,0.0


Queremos que elimine los que tenga un NaN

In [53]:
dif_abajo = test2 - redondeo_abajo

In [54]:
nan1 = dif_arriba[(dif_arriba != 0).all(1)]
nan1

,Fracción Volumen,Radio (nm),Espesor de la película (nm),zeros


In [55]:
def evaluar_red_neuronal(y_pred, y_true):
    # Convertir los DataFrames a numpy arrays
    y_pred = y_pred.values
    y_true = y_true.values
    
    # Calcular el MSE y el R2
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Imprimir los resultados
    print("MSE: ", mse)
    print("R2: ", r2)
    
    # Devolver el MSE y el R2 como una tupla
    return (mse, r2)

In [56]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Espesor de la película (nm)'], test2['Espesor de la película (nm)'])


MSE:  249.80208333333334
R2:  -0.08938671118283525


In [57]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Fracción Volumen'], test2['Fracción Volumen'])


MSE:  18.51875
R2:  -0.19159433231655165


In [58]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Radio (nm)'], test2['Radio (nm)'])


MSE:  106.77421875
R2:  -0.5408232160122908


# Ahora invertimos la red, de input será Fracción vol, espesor y radio. Output longitud de onda e Índice de absorción

In [109]:
# Cargar DataFrame
df = train_df
# Separar datos de entrada y salida
Y = df[['Índice de absorción', 'Longitud de onda']]
X = df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [110]:
# Convertimos los dataframes en numpy arrays
X = np.array(X)
y = np.array(Y)

In [111]:
# Creamos el modelo de la red neuronal
model = Sequential()
model.add(Dense(64, input_dim=3, activation='relu'))
model.add(Dense(2, activation='linear'))

In [112]:
model.compile(loss='mean_absolute_error', optimizer='adam')

In [113]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
960/960 [==============================] - 1s 864us/step - loss: 118.2328
Epoch 2/10
960/960 [==============================] - 1s 804us/step - loss: 80.6222
Epoch 3/10
960/960 [==============================] - 1s 744us/step - loss: 80.3896
Epoch 4/10
960/960 [==============================] - 1s 740us/step - loss: 80.2305
Epoch 5/10
960/960 [==============================] - 1s 735us/step - loss: 80.1138
Epoch 6/10
960/960 [==============================] - 1s 737us/step - loss: 80.0525
Epoch 7/10
960/960 [==============================] - 1s 818us/step - loss: 79.9665
Epoch 8/10
960/960 [==============================] - 1s 770us/step - loss: 79.9063
Epoch 9/10
960/960 [==============================] - 1s 835us/step - loss: 79.8122
Epoch 10/10
960/960 [==============================] - 1s 779us/step - loss: 79.7307


In [114]:
arr = test_df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [115]:
y_pred = model.predict(arr)
print(y_pred)

240/240 [==============================] - 0s 740us/step
[[-1.3513175e-01  6.3289026e+02]
 [-9.7930826e-02  6.4376154e+02]
 [-9.7930826e-02  6.4376154e+02]
 ...
 [-1.3782874e-01  6.3432886e+02]
 [-8.9538492e-02  6.4093329e+02]
 [-1.2675467e-01  6.5221808e+02]]


In [116]:
predicciones = pd.DataFrame(y_pred)
predicciones

,0,1
0,-0.135132,632.890259
1,-0.097931,643.761536
2,-0.097931,643.761536
3,-0.118999,624.258362
4,-0.017927,362.937195
...,...,...
7675,-0.129764,630.012939
7676,-0.181057,631.204590
7677,-0.137829,634.328857
7678,-0.089538,640.933289


# Renombrando las columnas

In [117]:
predicciones.columns = ['Índice de absorción', 'Longitud de onda']
predicciones

,Índice de absorción,Longitud de onda
0,-0.135132,632.890259
1,-0.097931,643.761536
2,-0.097931,643.761536
3,-0.118999,624.258362
4,-0.017927,362.937195
...,...,...
7675,-0.129764,630.012939
7676,-0.181057,631.204590
7677,-0.137829,634.328857
7678,-0.089538,640.933289


# Redondeos

In [118]:
redondeo_arriba = predicciones.copy()
redondeo_arriba['Longitud de onda'] = redondeo_arriba['Longitud de onda'].round()
redondeo_arriba

,Índice de absorción,Longitud de onda
0,-0.135132,633.0
1,-0.097931,644.0
2,-0.097931,644.0
3,-0.118999,624.0
4,-0.017927,363.0
...,...,...
7675,-0.129764,630.0
7676,-0.181057,631.0
7677,-0.137829,634.0
7678,-0.089538,641.0


In [119]:
redondeo_ceil = predicciones.copy()
redondeo_ceil['Longitud de onda'] = np.ceil(redondeo_ceil['Longitud de onda'])
redondeo_ceil

,Índice de absorción,Longitud de onda
0,-0.135132,633.0
1,-0.097931,644.0
2,-0.097931,644.0
3,-0.118999,625.0
4,-0.017927,363.0
...,...,...
7675,-0.129764,631.0
7676,-0.181057,632.0
7677,-0.137829,635.0
7678,-0.089538,641.0


In [120]:
redondeo_abajo = predicciones.copy()
redondeo_abajo['Longitud de onda'] = np.floor(redondeo_abajo['Longitud de onda'])
redondeo_abajo

,Índice de absorción,Longitud de onda
0,-0.135132,632.0
1,-0.097931,643.0
2,-0.097931,643.0
3,-0.118999,624.0
4,-0.017927,362.0
...,...,...
7675,-0.129764,630.0
7676,-0.181057,631.0
7677,-0.137829,634.0
7678,-0.089538,640.0


# Probando la accurancy del modelo

Primero lo haremos por cada parámetro. Se le restará los valores predichos por la red a los valores originales de test. Se contarán el número de ceros y ese será lo acertada que es nuestra red. De tener valores inferiores a 80% en número de ceros este rango se incrementará hasta llegar a un valor que el 80% de los valores sean menores a este

# Longitud de onda

Redondeo abajo

In [121]:
redondeo_abajo_long = redondeo_abajo['Longitud de onda'].to_numpy()
test_df_long = test_df['Longitud de onda'].to_numpy()
dif = redondeo_abajo_long - test_df_long 
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en longitud de onda")

presición del  0.2604166666666667 %. Con el redondeo para abajo en longitud de onda


Redondeo ceil

In [122]:
redondeo_ceil_long = redondeo_ceil['Longitud de onda'].to_numpy()
test_df_long = test_df['Longitud de onda'].to_numpy()
dif = redondeo_ceil_long - test_df_long
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo ceil en longitud de onda")

presición del  0.14322916666666666 %. Con el redondeo ceil en longitud de onda


Redondeo hacia arriba

In [123]:
redondeo_arriba_long = redondeo_arriba['Longitud de onda'].to_numpy()
test_df_long = test_df['Longitud de onda'].to_numpy()
dif = redondeo_arriba_long - test_df_long
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo arriba en longitud de onda")

presición del  0.22135416666666666 %. Con el redondeo arriba en longitud de onda


# Índice de absorción

In [74]:
redondeo_abajo_IA = redondeo_abajo['Índice de absorción'].to_numpy()
test_df_IA = test_df['Índice de absorción'].to_numpy()
dif = redondeo_abajo_IA - test_df_IA 
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en Índice de absorción")

presición del  0.0 %. Con el redondeo para abajo en Índice de absorción


In [75]:
# Define la entrada
input_layer = Input(shape=(2,))

# Capas ocultas compartidas
hidden1 = Dense(64, activation='relu')(input_layer)
hidden2 = Dense(32, activation='relu')(hidden1)

# Ramas para cada salida
branch1 = Dense(16, activation='relu')(hidden2)
output1 = Dense(1, activation='linear', name='fraccion_volumen')(branch1)

branch2 = Dense(16, activation='relu')(hidden2)
output2 = Dense(1, activation='linear', name='radio')(branch2)

branch3 = Dense(16, activation='relu')(hidden2)
output3 = Dense(1, activation='linear', name='espesor')(branch3)

# Define el modelo
model = Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compila el modelo
model.compile(loss='mean_squared_error', optimizer='adam')

NameError: name 'Input' is not defined